In [1]:
import connection
import pathlib

# location of PAT
pat = pathlib.Path('/Users/aniket/ws/tokens/qa.token')

# legacy JDBC driver location
driver = pathlib.Path('/Users/aniket/downloads/dremio-jdbc-driver-24.3.2-202401241821100032-d2d8a497.jar')


## Connect using Legacy JDBC

In [2]:
# project ID for the project to connect to
project_id = 'ad66f49c-86bb-42d6-b7cd-ffaceafafc14'
params = connection.JdbcConnectionParams(connection.Region.NA, pat.read_text(), connection.JdbcConnectionParams.TokenType.PAT,
                                         project_id=project_id, driver_jar=driver)
jconn = params.connect()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


java.sql.SQLException: java.sql.SQLException: Failure in connecting to Dremio: cdjd.com.dremio.exec.rpc.RpcException: HANDSHAKE_VALIDATION : Status: AUTH_FAILED, Error Id: 43aaaaac-e26c-45fc-98f9-3fe3c996764f, Error message: Error authenticating connection Invalid user credentials, user 

In [8]:
# work with sample data
with jconn.cursor() as cur:
    cur.execute(
        'select pickup_datetime, passenger_count, total_amount from Samples."samples.dremio.com"."NYC-taxi-trips-iceberg" limit 10')
    while row := cur.fetchone():
        print(row)

('2013-02-10 21:59:00', 2, 9.5)
('2013-02-10 22:00:00', 6, 7.0)
('2013-02-10 21:57:00', 1, 8.62)
('2013-02-10 21:39:00', 1, 57.3)
('2013-02-08 09:44:00', 1, 15.45)
('2013-02-10 22:01:00', 1, 8.3)
('2013-02-10 21:47:00', 1, 29.0)
('2013-02-08 09:47:00', 1, 12.5)
('2013-02-08 09:53:00', 1, 10.7)
('2013-02-08 09:54:00', 5, 5.0)


## Connect using ADBC

This has the additional benefit of integrating directly with pandas

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

params = connection.AdbcConnectionParams(connection.Region.NA, pat.read_text(), project_id=project_id)
conn = params.connect()

In [4]:
pd.read_sql(
    'select * from Samples."samples.dremio.com"."NYC-taxi-trips-iceberg" limit 10', conn)

DatabaseError: Execution failed on sql 'select * from Samples."samples.dremio.com"."NYC-taxi-trips-iceberg" limit 10': UNKNOWN: [FlightSQL] [FlightSQL]  (Unknown; Prepare)